In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
from pandas.io.json import json_normalize

In [7]:
df = pd.read_json("train.json",
                 #lines = True,
                 orient = 'columns',
                 typ = 'series')
print(df.shape)
#df = df.rename_axis('text', axis=0)
print(df.head())

(4152659,)
 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.363779 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n    normal
 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.527847 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n    normal
 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.675872 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n    normal
 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.823719 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n    normal
 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.982731 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n    normal
dtype: object


In [8]:
import string
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    stemmer = PorterStemmer()

    text_stemmed = [stemmer.stem(y) for y in text_filtered]

    final_string = ' '.join(text_stemmed)

    return final_string

In [9]:
msgs = [clean_string(v) for v in df.index]
data = pd.DataFrame()
data['msg'] = msgs
labels = [0 if df[v] == 'abnormal' else 1 for v in df.index]
data['labels'] = labels

In [10]:
data.head()
X = data['msg']
y = data['labels']

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

print('Training Data Shape:', X_train.shape)
print('Testing Data Shape: ', X_test.shape)

Training Data Shape: (2906861,)
Testing Data Shape:  (1245798,)


In [21]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LogisticRegression()),])
text_clf.fit(X, y)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LogisticRegression())])

In [22]:
test_data = pd.read_csv('test.csv')
testData=test_data[' Log']
testing = text_clf.predict(testData)

In [ ]:
out = pd.DataFrame()
out['ID'] = [id for id in test_data['ID']]
out[' Label'] = ['abnormal' if v == 0 else 'normal' for v in testing]
out.to_csv('submission.csv', index = False)